# context 

This script performs the final prediction using the LSTM model selected during the model‐selection phase. It also implements our chosen strategies by fetching fresh data directly from the API, making the script fully self‐contained and runnable at any time to have recent strategy.

# Imports

In [ ]:
%pip install tensorflow

In [ ]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.offsets import BDay

from pyspark.sql import SparkSession, Window, Row
from pyspark.sql.types import StructType, StructField, DateType, DoubleType
from pyspark.sql.functions import (
    col, to_date, lag, avg, stddev, log, when, current_date,
    date_sub, add_months, expr, row_number, dayofweek, lit,
    pandas_udf, PandasUDFType
)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
symbol =  "AAPL"
api_key = "UDEJQ3FLFIY9BVVT"

In [ ]:
spark = SparkSession.builder.appName("AAPL_Prediction").getOrCreate()

In [ ]:
def get_alpha_vantage_data(symbol, key):
    function = 'TIME_SERIES_DAILY'
    output_size = 'full'
    url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&outputsize={output_size}&apikey={key}'

    r = requests.get(url)
    data = r.json()

    if 'Time Series (Daily)' in data:
        df = pd.DataFrame(data['Time Series (Daily)']).T
        df.index = pd.to_datetime(df.index)
        df.sort_index(inplace=True)
        df.columns = [col.split('. ')[1] for col in df.columns]
        df = df.apply(pd.to_numeric)
        df = df[df.index >= datetime.now() - timedelta(days=365*20)]

        # Filter for data up to 18th of May 2025
        cutoff_date = pd.Timestamp('2025-05-18')
        df = df[df.index <= cutoff_date]

        df.reset_index(inplace=True)
        df.rename(columns={'index': 'Date'}, inplace=True)
        return spark.createDataFrame(df)
    else:
        raise Exception("Alpha Vantage API Error")

In [ ]:
spark_df = get_alpha_vantage_data(symbol, api_key)

In [ ]:
splits = {
    '2005-02-28': 2, # two-for-one stock split on 28 February 2005
    '2014-06-09': 7, #  seven-for-one stock split on 9th of June 2014
    '2020-08-31': 4. #  four-for-one stock split on 31st of August 2020
}

for split_date_str, stock_split in splits.items():
    split_date = datetime.strptime(split_date_str, "%Y-%m-%d")
    
    # Adjust prices before split date
    spark_df = spark_df.withColumn(
        'open',
        when(col('Date') < lit(split_date), col('open') / stock_split).otherwise(col('open'))
    ).withColumn(
        'high',
        when(col('Date') < lit(split_date), col('high') / stock_split).otherwise(col('high'))
    ).withColumn(
        'low',
        when(col('Date') < lit(split_date), col('low') / stock_split).otherwise(col('low'))
    ).withColumn(
        'close',
        when(col('Date') < lit(split_date), col('close') / stock_split).otherwise(col('close'))
    ).withColumn(
        'volume',
        when(col('Date') < lit(split_date), col('volume') * stock_split).otherwise(col('volume'))
    )



# Final Prediction

We use the LSTM model chosen during model selection. Because Spark ML doesn’t support LSTMs, we:

1. Convert the Spark DataFrame to pandas  
2. Run the LSTM predictions in pandas  
3. Convert the pandas results back into a Spark DataFrame  

**What’s changed**

- **Larger window size**  
  The model now looks further back in time to capture more variation.

- **Dropout layer**  
  Added dropout to improve generalization and reduce overfitting.

- **One-shot dense output**  
  A Dense layer predicts the entire forecast horizon at once, avoiding rolling forecasts.

 

In [ ]:
# represent the number of business days in a year
window_size = 252
horizon     = 252

spark_df = spark_df.select("date", "close")
spark_df = spark_df.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))
training_pdf = spark_df.toPandas()

train_vals   = training_pdf["close"].values.reshape(-1,1)
scaler       = MinMaxScaler()
scaled_all   = scaler.fit_transform(train_vals).flatten()

In [ ]:
# Create sliding windows
X, Y = [], []
for i in range(len(scaled_all) - window_size - horizon + 1):
    X.append(scaled_all[i : i + window_size])
    Y.append(scaled_all[i + window_size : i + window_size + horizon])
X = np.array(X).reshape(-1, window_size, 1)  # (samples, window_size, 1)
Y = np.array(Y)                              # (samples, horizon)

# model definition 
model = Sequential([
    LSTM(64, input_shape=(window_size, 1)),
    Dropout(0.2),
    Dense(horizon)      # predict `horizon` steps at once
])
model.compile(optimizer="adam", loss="mse", metrics=["mse"])
history = model.fit(
    X, Y,
    epochs=10,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# Predict on the last window
last_window = scaled_all[-window_size:].reshape(1, window_size, 1)
pred_scaled = model.predict(last_window)[0]

# 5) Inverse scale forecasts
pred_prices = scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()

# 6) Prepare dates
last_date    = training_pdf["date"].iloc[-1]
future_dates = pd.bdate_range(start=last_date + BDay(1), periods=horizon)

# 7) Plot only the last year of actual data
one_year_ago = last_date - pd.DateOffset(years=1)
mask_recent  = training_pdf["date"] >= one_year_ago

plt.figure(figsize=(10, 5))
plt.plot(
    training_pdf.loc[mask_recent, "date"],
    training_pdf.loc[mask_recent, "close"],
    label="Historical (Last 1 Year)"
)
plt.plot(future_dates, pred_prices, label="1-Year Direct Forecast")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.legend()
plt.tight_layout()
plt.show()